In [ ]:
!pip install psycopg2-binary pandas

In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_smfWpB5zj2iX",  # use the same as in your psql command
    host="ep-small-firefly-ade7s4w8-pooler.c-2.us-east-1.aws.neon.tech",  # same host
    port="5432",
    sslmode="require"  # ensure SSL is enabled
)

cur = conn.cursor()
print("Connected successfully!")


Connected successfully!


In [ ]:

# SQL statement to create Classes table
create_classes_sql = """
CREATE TABLE IF NOT EXISTS Classes (
    class TEXT PRIMARY KEY,
    type TEXT,
    country TEXT,
    numGuns INT,
    bore REAL,
    displacement INT
);
"""
# Execute SQL
cur.execute(create_classes_sql)
conn.commit()
print("Classes table created successfully!")


Classes table created successfully!


In [ ]:
create_battles_sql = """
CREATE TABLE IF NOT EXISTS Battles (
    name TEXT PRIMARY KEY,
    date DATE
);
"""
cur.execute(create_battles_sql)
conn.commit()
print("Battles table created successfully!")


Battles table created successfully!


In [ ]:
insert_classes_sql = """
INSERT INTO Classes (class, type, country, numGuns, bore, displacement) VALUES
('Bismarck', 'bb', 'Germany', 8, 15, 42000),
('Iowa', 'bb', 'USA', 9, 16, 46000),
('Kongo', 'be', 'Japan', 8, 14, 32000),
('North Carolina', 'bb', 'USA', 9, 16, 37000),
('Renown', 'be', 'Gt. Britain', 6, 15, 32000),
('Revenge', 'bb', 'Gt. Britain', 8, 15, 29000),
('Tennessee', 'bb', 'USA', 12, 14, 32000),
('Yamato', 'bb', 'Japan', 9, 18, 65000);
"""

cur.execute(insert_classes_sql)
conn.commit()
print("Inserted data into Classes successfully!")



Inserted data into Classes successfully!


In [ ]:
insert_battles_sql = """
INSERT INTO Battles (name, date) VALUES
('Denmark Strait', '1941-05-24'),
('Guadalcanal', '1942-11-15'),
('North Cape', '1943-12-26'),
('Surigao Strait', '1944-10-25');
"""

cur.execute(insert_battles_sql)
conn.commit()
print("Inserted data into Battles successfully!")


Inserted data into Battles successfully!


In [ ]:
# Query 1a: Classes with at least 10 guns
cur.execute("SELECT class, country FROM Classes WHERE numGuns >= 10;")
rows = cur.fetchall()
print("\n-- Query 1a: Classes with >=10 guns --")
display(pd.DataFrame(rows, columns=["class","country"]))

# Query 1e: Classes starting with 'R'
cur.execute("SELECT class FROM Classes WHERE class LIKE 'R%';")
rows = cur.fetchall()
print("\n-- Query 1e: Classes starting with 'R' --")
display(pd.DataFrame(rows, columns=["class"]))



-- Query 1a: Classes with >=10 guns --


,class,country
0,Tennessee,USA



-- Query 1e: Classes starting with 'R' --


,class
0,Renown
1,Revenge


In [ ]:
# Query 2a: Classes heavier than 35,000 tons
cur.execute("SELECT class, displacement FROM Classes WHERE displacement > 35000;")
rows = cur.fetchall()
print("\n-- Query 2a: Classes heavier than 35,000 tons --")
display(pd.DataFrame(rows, columns=["class","displacement"]))

# Query 2b: Battles after Jan 1, 1942
cur.execute("SELECT name, date FROM Battles WHERE date > '1942-01-01';")
rows = cur.fetchall()
print("\n-- Query 2b: Battles after Jan 1, 1942 --")
display(pd.DataFrame(rows, columns=["name","date"]))


-- Query 2a: Classes heavier than 35,000 tons --


,class,displacement
0,Bismarck,42000
1,Iowa,46000
2,North Carolina,37000
3,Yamato,65000



-- Query 2b: Battles after Jan 1, 1942 --


,name,date
0,Guadalcanal,1942-11-15
1,North Cape,1943-12-26
2,Surigao Strait,1944-10-25


In [ ]:
cur.execute("""
SELECT c.class, c.numGuns, b.name AS battle, b.date
FROM Classes c
CROSS JOIN Battles b
WHERE c.numGuns > 8 AND b.date > '1942-01-01';
""")
rows = cur.fetchall()
print("\n-- Challenge Query: Classes with >8 guns & battles after 1942 --")
display(pd.DataFrame(rows, columns=["class","numGuns","battle","date"]))


-- Challenge Query: Classes with >8 guns & battles after 1942 --


,class,numGuns,battle,date
0,Iowa,9,Guadalcanal,1942-11-15
1,North Carolina,9,Guadalcanal,1942-11-15
2,Tennessee,12,Guadalcanal,1942-11-15
3,Yamato,9,Guadalcanal,1942-11-15
4,Iowa,9,North Cape,1943-12-26
5,North Carolina,9,North Cape,1943-12-26
6,Tennessee,12,North Cape,1943-12-26
7,Yamato,9,North Cape,1943-12-26
8,Iowa,9,Surigao Strait,1944-10-25
9,North Carolina,9,Surigao Strait,1944-10-25
